# Einführung

Im diesem Teil des Tutorials zeige ich dir die ersten Schritte mit LangChain. Dabei installieren wir die notwendigen Bibliotheken und schreiben den allerersten Basiscode um mit einem Sprachmodell wie ChatGPT (OpenAI) zu kommunizieren.


# Warum LangChain?

Die Anbieter von KIs (Sprachmodellen/LLMs) bieten häufig eigene APIs an, mit denen man mit die KI in die eigene Anwendung integrieren kann. So kann man z.B. mit der API von OpenAI bereits sehr einfach die ersten Aufrufe machen.
Damit wir uns nicht an die API der jeweiligen Anbieter binden müssen, bietet uns LangChain uns eine einheitliche API um mit allen möglichen LLMs zu kommunizieren. Je nach Anwendungsfall können wir sogar die verschiedenen LLMs in einer Anwendung verwenden. So können wir mit einem Open-Source-Modell sog. Embeddings erstellen (mehr dazu in einem anderen Kapitel) und die Chat-Funktion von ChatGPT (OpenAI) nutzen. Wir können auch jederzeit das verwendete LLM austauschen, ohne den Code neu schreiben zu müssen.
Zusätzlich dazu bietet LangChain vielfältige weitere Möglichkeiten. Mit den PromptTemplates können wir die Erstellung und Verwaltung von Prompts (Eingaben für die LLMs) bewerkstelligen, ohne irgendwelche Strings zusammen zu basteln. Wir können mit den Output Parsern die Ausgaben der LLMs maschinell auswerten und z.B. uns das Ergebnis als JSON zurück geben zu lassen, mit dem wir in unserem Programm arbeiten können. Spannend wird es spätestens dann, wenn wir die Agenten "erwecken". D.h. wir geben der KI Werkzeuge, mit denen die KI mehr als nur Texte erzeugen kann. Z.B. Informationen beschaffen, Berechnungen durchführen uvm.

# Installation notwendiger Bibliotheken

Für die meisten Beispiele dieses Tutorial brauchen wir einige Bibliotheken. Wir brauchen natürlich die LangChain-Bibliothek. Daneben brauchen wir die OpenAI Bibliothek um mit ChatGPT zu kommunizieren (direkt aber auch unter Verwendung von LangChain). Dann benötigen wir außerdem Python Dotenv damit wir den notwendigen API-Key aus einer .env Datei laden können. Das hat den Vorteil, dass wir den geheimen API-Key nicht direkt im Code hinterlegen müssen und statt dessen in der .env Datei speichern. Diese Datei wird nicht im Git-Repository eingecheckt (in .gitignore ist diese Datei hinterlegt) und wir somit nicht aus versehen unseren API-Key veröffentlichen.

Führe zunächst folgenden Befehl aus:

In [1]:
%pip install langchain openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Beispiel: Direkte Verwendung der OpenAI-Bibliothek

Bevor wir mit LangChain starten, erstmal ein Beispiel dafür, wie man die OpenAI (ChatGPT) API direkt nutzen. Das ermöglicht uns die Verwendung zu vergleichen.

In [5]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [{"role": "user", "content": "Erkläre mir bitte, wie ich gute Eingaben für ChatGPT schreibe."}]

response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

print(response)

{
  "id": "chatcmpl-87OHL8PL3oWWQr56dXBW64ri7GFgb",
  "object": "chat.completion",
  "created": 1696772315,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Um gute Eingaben f\u00fcr ChatGPT zu schreiben, folgen hier einige Empfehlungen:\n\n1. Sei spezifisch: Versuche, klare und pr\u00e4zise Anweisungen zu geben, damit der Chatbot die gew\u00fcnschte Aufgabe oder das gew\u00fcnschte Ergebnis besser verstehen kann.\n\n2. Sei pr\u00e4zise mit deinen Fragen: Stelle konkrete Fragen, anstatt allgemeine Aussagen zu machen. Es hilft dem Chatbot, gezielte Antworten zu geben.\n\n3. Vermeide doppelte Formulierungen: Wiederhole nicht denselben Satz oder dieselbe Information in verschiedenen Eingaben. Dadurch k\u00f6nnte der Chatbot verwirrt werden.\n\n4. Verwende Kontext: Um eine reibungslose Konversation zu gew\u00e4hrleisten, gib dem Chatbot klare Anhaltspunkte zur vorherigen Konversation oder zum Kontext

Wir sehen, dass wir mit wenigen Zeilen Code eine Anfrage an ChatGPT senden können. Dabei fallen ein paar Sachen auf. Das erste ist, dass wir eine Liste von Nachrichten (_messages_) an die API übergeben. Außerdem enthält die Antwort wesentlich mehr, als nur den reinen Text der Antwort.

Der Grund für die Liste der Nachrichten ist die, dass wir die Chat-Funktionalität nutzen. Der Server behält sich nicht die vorher eingegebenen Fragen und Antworten. D.h. man muss stets den gesamten Chat-Verlauf inkl. Antworten zurück übermitteln. Dadurch weiß die KI über welche Themen wir zuvor "gesprochen" haben. 
D.h. wenn ich jetzt eine Frage stellen möchte, die sich auf den bisherigen Chatverlauf bezieht, muss ich die Nachrichten-Liste erweitern:

In [6]:
# Die Antwort der KI muss als Nachricht in die Liste hinten angehängt werden
messages.append(response["choices"][0]["message"]);

# Die neue Frage kommt noch dazu
messages.append({"role": "user", "content": "Danke für die Erklärung. Kannst du bitte die wesentlichen Punkte als kurze Liste zusammen fassen"})

# Erneuter Aufruf
response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

print(response)

{
  "id": "chatcmpl-87OHZHS0IOI7F6Bjkd3oNhGUGMQul",
  "object": "chat.completion",
  "created": 1696772329,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Nat\u00fcrlich! Hier sind die wesentlichen Punkte als kurze Liste:\n\n1. Sei spezifisch.\n2. Stelle pr\u00e4zise Fragen.\n3. Vermeide doppelte Formulierungen.\n4. Verwende Kontext.\n5. Sei h\u00f6flich und klar.\n6. Experimentiere und iteriere.\n7. Vermeide voreingenommene oder unangemessene Fragen.\n\nDiese Punkte helfen dir dabei, gute Eingaben f\u00fcr ChatGPT zu erstellen und bessere Ergebnisse zu erzielen."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 507,
    "completion_tokens": 121,
    "total_tokens": 628
  }
}


In [8]:
# Schönere Ausgabe 

from IPython.display import display, Markdown
display(Markdown(response["choices"][0]["message"]['content']))

Natürlich! Hier sind die wesentlichen Punkte als kurze Liste:

1. Sei spezifisch.
2. Stelle präzise Fragen.
3. Vermeide doppelte Formulierungen.
4. Verwende Kontext.
5. Sei höflich und klar.
6. Experimentiere und iteriere.
7. Vermeide voreingenommene oder unangemessene Fragen.

Diese Punkte helfen dir dabei, gute Eingaben für ChatGPT zu erstellen und bessere Ergebnisse zu erzielen.

Damit könnten wir schon eine Komplette Chat-Anwendung programmieren. Einfach Eingabe vom Benutzer holen, der Liste der Nachrichten hinzufügen, Aufruf ChatGPT, Antwort hinzufügen und das Ganze nochmal wiederholen.

Lasst uns jetzt erste Schritte mit LangChain machen. Wir werden später sehen, wo uns LangChain hilft, diese Schritte zu automatisieren und wesentlich bequemer und übersichtlicher zu gestalten.

Wir nutzen jetzt LangChain um mit ChatGPT zu kommunizieren

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

llm = ChatOpenAI(model="gpt-3.5-turbo")

response = llm.predict("Spreche ich mit ChatGPT?")

print(response)